In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def load_all_data():
    """Carga todos los archivos JSON necesarios."""
    data = {
        'horarios_salas': load_json_file('../../agent_output/Horarios_salas.json'),
        'salas_input': load_json_file('../../agent_input/InputOfSala.json')
    }
    
    if None in data.values():
        print("Error: No se pudieron cargar todos los archivos necesarios")
        return None
    return data

def create_room_capacity_dict(salas_input):
    """Crea un diccionario con las capacidades de las salas."""
    try:
        return {room['Codigo']: room['Capacidad'] for room in salas_input}
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de las salas")
        return None

def categorize_capacity(current_capacity):
    """Categoriza la capacidad según los rangos definidos."""
    if current_capacity < 1.0:
        return 'bajo'
    elif current_capacity > 1.00:
        return 'sobre'
    else:
        return 'exacta'

def analyze_room_assignments(horarios_salas, room_capacities):
    """Analiza las asignaciones de cada sala."""
    capacity_stats = defaultdict(lambda: {'bajo': 0, 'exacta': 0, 'sobre': 0, 'total': 0})
    
    try:
        for sala in horarios_salas:
            codigo = sala['Codigo']
            
            for asignatura in sala['Asignaturas']:
                current_capacity = asignatura['Capacidad']
                capacity_stats[codigo]['total'] += 1
                category = categorize_capacity(current_capacity)
                capacity_stats[codigo][category] += 1
                
        return capacity_stats
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de asignaciones")
        return None

def create_capacity_dataframe(capacity_stats, room_capacities):
    """Crea un DataFrame con las estadísticas de capacidad."""
    try:
        df_stats = pd.DataFrame.from_dict(capacity_stats, orient='index')
        df_stats['room_capacity'] = df_stats.index.map(room_capacities)
        
        # Calcular porcentajes
        for col in ['bajo', 'exacta', 'sobre']:
            df_stats[f'{col}_pct'] = (df_stats[col] / df_stats['total'] * 100).round(2)
        
        # Crear rangos de capacidad
        df_stats['capacity_range'] = pd.cut(
            df_stats['room_capacity'], 
            bins=range(0, 91, 10), 
            labels=[f'{i}-{i+9}' for i in range(0, 81, 10)]
        )
        
        return df_stats
    except Exception as e:
        print(f"Error al crear el DataFrame: {str(e)}")
        return None

def create_summary_table(df_stats):
    """Crea una tabla resumen de las estadísticas de capacidad."""
    try:
        summary_table = pd.DataFrame({
            'Sala': df_stats.index,
            'Capacidad Total': df_stats['room_capacity'],
            'Bajo Capacidad (%)': df_stats['bajo_pct'],
            'Capacidad Exacta (%)': df_stats['exacta_pct'],
            'Sobre Capacidad (%)': df_stats['sobre_pct']
        })
        
        # Guardar tabla en CSV
        summary_table.to_csv('resumen_capacidad.csv', index=False)
        
        # Mostrar tabla formateada
        print("\nResumen de Capacidad por Sala:")
        print(summary_table.to_string(index=False))
        
        return summary_table
    except Exception as e:
        print(f"Error al crear la tabla resumen: {str(e)}")
        return None

def create_pie_chart(df_stats):
    """Crea el gráfico de torta de distribución global."""
    try:
        plt.figure(figsize=(10, 6))
        total_counts = df_stats[['bajo', 'exacta', 'sobre']].sum()
        plt.pie(total_counts, 
                labels=['Bajo Capacidad', 'Capacidad Exacta', 'Sobre Capacidad'],
                colors=['#90EE90', '#3CB371', '#CD5C5C'],
                autopct='%1.1f%%')
        plt.title('Distribución Global de Ocupación de Salas')
        plt.savefig('ocupacion_global.png')
        plt.close()
        return total_counts
    except Exception as e:
        print(f"Error al crear el gráfico de torta: {str(e)}")
        return None

def create_stacked_bar_chart(df_stats):
    """Crea el gráfico de barras apiladas por rango de capacidad."""
    try:
        # Crear análisis por rango
        range_analysis = df_stats.groupby('capacity_range', observed=True).agg({
            'bajo': 'sum',
            'exacta': 'sum',
            'sobre': 'sum'
        }).fillna(0)
        
        # Crear gráfico
        plt.figure(figsize=(12, 6))
        range_analysis.plot(kind='bar', stacked=True, color=['#90EE90', '#3CB371', '#CD5C5C'])
        plt.title('Distribución de Ocupación por Rango de Capacidad')
        plt.xlabel('Rango de Capacidad')
        plt.ylabel('Número de Asignaciones')
        plt.legend(title='Tipo de Ocupación', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig('ocupacion_por_rango.png', bbox_inches='tight')
        plt.close()
        
        return range_analysis
    except Exception as e:
        print(f"Error al crear el gráfico de barras: {str(e)}")
        return None

def save_results(df_stats, total_counts, range_analysis):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_stats.to_dict(orient='index'),
            'global_stats': {
                'bajo_capacidad': float(total_counts['bajo'] / total_counts.sum() * 100),
                'capacidad_exacta': float(total_counts['exacta'] / total_counts.sum() * 100),
                'sobre_capacidad': float(total_counts['sobre'] / total_counts.sum() * 100)
            },
            'range_analysis': range_analysis.to_dict()
        }
        
        with open('metricas_capacidad.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    data = load_all_data()
    if data is None:
        return
    
    # 2. Crear diccionario de capacidades
    room_capacities = create_room_capacity_dict(data['salas_input'])
    if room_capacities is None:
        return
    
    # 3. Analizar asignaciones
    capacity_stats = analyze_room_assignments(data['horarios_salas'], room_capacities)
    if capacity_stats is None:
        return
    
    # 4. Crear DataFrame
    df_stats = create_capacity_dataframe(capacity_stats, room_capacities)
    if df_stats is None:
        return
    
    # 5. Crear tabla resumen
    summary_table = create_summary_table(df_stats)
    if summary_table is None:
        return
    
    # 6. Crear gráficos
    total_counts = create_pie_chart(df_stats)
    if total_counts is None:
        return
    
    range_analysis = create_stacked_bar_chart(df_stats)
    if range_analysis is None:
        return
    
    # 7. Guardar resultados
    if save_results(df_stats, total_counts, range_analysis):
        print("\nAnálisis completado exitosamente")
    else:
        print("Error al completar el análisis")

if __name__ == "__main__":
    main()


Resumen de Capacidad por Sala:
 Sala  Capacidad Total  Bajo Capacidad (%)  Capacidad Exacta (%)  Sobre Capacidad (%)
 A106               18                25.0                  8.33                66.67
ANT21               22                80.0                  0.00                20.00
 LPIR               24               100.0                  0.00                 0.00

Análisis completado exitosamente


<Figure size 1200x600 with 0 Axes>